In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/CarloSgaravatti/Recsys_Challenge_2023.git

Cloning into 'Recsys_Challenge_2023'...
remote: Enumerating objects: 5104, done.
remote: Counting objects: 100% (293/293), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 5104 (delta 240), reused 163 (delta 134), pack-reused 4811
Receiving objects: 100% (5104/5104), 2.79 GiB | 28.15 MiB/s, done.
Resolving deltas: 100% (4066/4066), done.
Updating files: 100% (463/463), done.


In [2]:
! pip install PyGithub requests

  Obtaining dependency information for PyGithub from https://files.pythonhosted.org/packages/be/04/810d131be173cba445d3658a45512b2b2b3d0960d52c4a300d6ec5e00f52/PyGithub-2.1.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 711.7 kB/s eta 0:00:00


In [3]:
# ! cd /kaggle/working/Recsys_Challenge_2023 && python run_compile_all_cython.py

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import random
import tqdm
import gc
import time
import os
import shutil
import optuna
import scipy.sparse as sps

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv
/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv


In [5]:
%cd /kaggle/working/Recsys_Challenge_2023

/kaggle/working/Recsys_Challenge_2023


In [6]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_SVDpp_Cython, MatrixFactorization_AsySVD_Cython, MatrixFactorization_WARP_Cython
from Recommenders.MatrixFactorization.PyTorch.MF_MSE_PyTorch import MF_MSE_PyTorch
from Recommenders.BaseMatrixFactorizationRecommender import BaseMatrixFactorizationRecommender
from Recommenders.Incremental_Training_Early_Stopping import Incremental_Training_Early_Stopping

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

In [7]:
from github import Github
from github import Auth

auth = Auth.Token(token)
g = Github(auth=auth)

repo = None
for r in g.get_user().get_repos():
    if r.name == 'Recsys_Challenge_2023':
        repo = r
        print('Repository found')

Repository found


In [8]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    try:
        contents = repo.get_contents(filepath_github)
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(contents.path, commit_message, file.read(), contents.sha)
    except Exception:
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(filepath_github, commit_message, file.read())

In [9]:
config = {
    'tune_parameters': True,
    'database_path': '/kaggle/working/tuning_lightgcn.db',
    'copy_prev_best_params': False,
    'save_github': True
}

In [10]:
try:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/tuning_lightgcn.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

In [11]:
train_data = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv')
user_ids_test = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv')

In [12]:
URM_all = sps.csr_matrix((train_data.data.values,
                          ((train_data.row - 1).values, (train_data.col - 1).values)),
                        shape=(train_data.row.max(), train_data.col.max()))
URM_all

<13024x22347 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [13]:
URM_train, URM_test_complete = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_validation, URM_test = split_train_in_two_percentage_global_sample(URM_test_complete, train_percentage = 0.8)

In [14]:
import tensorflow as tf
import gc


class LightGCNRecommender(BaseMatrixFactorizationRecommender, Incremental_Training_Early_Stopping):
    
    RECOMMENDER_NAME = "LightGCNRecommender"
    AVAILABLE_OPTIMIZERS = ["adam", "sgd", "rmsprop", "adagrad"]
    AVAILABLE_LOSSES = ["mse", "xentropy"] # todo: add bpr
    
    def __init__(self, URM_train):
        super(LightGCNRecommender, self).__init__(URM_train)
        
        # building the tensorflow sparse tensor, .T is done because an array of (row, col) is needed
        URM_train_coo = self.URM_train.tocoo()
        self.URM_train_tensor = tf.sparse.SparseTensor(
            np.array([URM_train_coo.row, URM_train_coo.col]).T, URM_train_coo.data, URM_train_coo.shape
        )
        
    def fit(
        self, 
        epochs=100, 
        num_layers=2,
        embedding_size=10,
        learning_rate=1e-3,
        init_mean=0.0,
        init_std=0.01,
        optimizer='adam',
        loss='mse',
        l2_user=1e-2,
        l2_item=1e-2,
        **earlystopping_kwargs
    ):
        
        if optimizer not in self.AVAILABLE_OPTIMIZERS:
            raise ValueError("Value for 'optimizer' not recognized. Acceptable values are {}, provided was '{}'".format(self.AVAILABLE_OPTIMIZERS, optimizer))
            
        if loss not in self.AVAILABLE_LOSSES:
            raise ValueError("Value for 'loss' not recognized. Acceptable values are {}, provided was '{}'".format(self.AVAILABLE_LOSSES, loss))

        self.num_layers = num_layers
        self.embedding_size = embedding_size
        self.learning_rate = learning_rate
        
        self._build_optimizer(optimizer)
        self._build_loss(loss)
        
        # the only variables are the embeddings at layer 0
        self.user_embeddings_initial_step = tf.Variable(tf.random.normal([self.n_users, self.embedding_size], mean=init_mean, stddev=init_std))
        self.item_embeddings_initial_step = tf.Variable(tf.random.normal([self.n_items, self.embedding_size], mean=init_mean, stddev=init_std))
        
        self.user_embeddings_reg = tf.keras.regularizers.l2(l2_user)(self.user_embeddings_initial_step)
        self.item_embeddings_reg = tf.keras.regularizers.l2(l2_item)(self.item_embeddings_initial_step)
        
        # neighborhood sizes of the items
        self.N_i = tf.sparse.reduce_sum(self.URM_train_tensor, axis=0)
        self.N_i = tf.sqrt(self.N_i + 1e-6)
        self.N_i_inv = 1.0 / self.N_i
        self.N_i_inv = tf.reshape(self.N_i_inv, shape=[-1, 1])
        
        # neighborhood sizes of the users
        self.N_u = tf.sparse.reduce_sum(self.URM_train_tensor, axis=1)
        self.N_u = tf.sqrt(self.N_u  + 1e-6)
        self.N_u_inv = 1.0 / self.N_u
        self.N_u_inv = tf.reshape(self.N_u_inv, shape=[-1, 1])
        
        self._train_with_early_stopping(epochs,
                                        algorithm_name = self.RECOMMENDER_NAME,
                                        **earlystopping_kwargs)

        # the final user and items factors, obtained by applying convolutions for num_layers times
        self.USER_factors = self.USER_factors_best
        self.ITEM_factors = self.ITEM_factors_best
        
    def _build_optimizer(self, optimizer):
        # todo: consider adding the other parameters of the optimizers
        if optimizer == 'adam':
            self.optimizer = tf.optimizers.Adam(learning_rate=self.learning_rate)
        elif optimizer == 'rmsprop':
            self.optimizer = tf.optimizers.RMSprop(learning_rate=self.learning_rate)
        elif optimizer == 'sgd':
            self.optimizer = tf.optimizers.SGD(learning_rate=self.learning_rate)
        else:
            self.optimizer = tf.optimizers.Adagrad(learning_rate=self.learning_rate)
            
    def _build_loss(self, loss):
        if loss == 'mse':
            self.loss_function = tf.losses.mean_squared_error
        else:
            self.loss_function = tf.losses.binary_crossentropy
    
    def _run_epoch(self, num_epoch):
        
        with tf.GradientTape(persistent=True) as tape:
            user_embeddings = self.user_embeddings_initial_step
            item_embeddings = self.item_embeddings_initial_step

            # k steps (layers) of convolutions
            for _ in range(self.num_layers):
                # computing the embeddings at step k+1 using the embedding at step k
                # this means that user embeddings have to be updated only at the end
                user_embeddings_update = self._graph_convolution_user(item_embeddings)
                item_embeddings_update = self._graph_convolution_item(user_embeddings)
                
                user_embeddings = user_embeddings_update
                item_embeddings = item_embeddings_update
                
            user_item_scores = tf.matmul(user_embeddings, item_embeddings, transpose_b=True)
            
            regularization = self.user_embeddings_reg + self.item_embeddings_reg
            loss = self.loss_function(tf.sparse.to_dense(self.URM_train_tensor), user_item_scores) + regularization

        user_gradients = tape.gradient(loss, self.user_embeddings_initial_step)
        item_gradients = tape.gradient(loss, self.item_embeddings_initial_step)

        self.optimizer.apply_gradients(zip([user_gradients, item_gradients], [self.user_embeddings_initial_step, self.item_embeddings_initial_step]))
        
        self._update_factors()
        gc.collect()

        del tape
        
    def _graph_convolution_user(self, item_embeddings):
        normalized_embeddings = item_embeddings * self.N_i_inv

        conv_output = tf.sparse.sparse_dense_matmul(self.URM_train_tensor, normalized_embeddings)
        return conv_output * self.N_u_inv
    
    def _graph_convolution_item(self, user_embeddings):
        normalized_embeddings = user_embeddings * self.N_u_inv

        conv_output = tf.sparse.sparse_dense_matmul(tf.sparse.transpose(self.URM_train_tensor), normalized_embeddings)
        return conv_output * self.N_i_inv
        
    def _update_factors(self):
        self.USER_factors = self.user_embeddings_initial_step.numpy().copy()
        self.ITEM_factors = self.item_embeddings_initial_step.numpy().copy()
        
        for _ in range(self.num_layers):
            USER_factors = self._graph_convolution_user(self.ITEM_factors).numpy()
            ITEM_factors = self._graph_convolution_item(self.USER_factors).numpy()
            
            self.USER_factors = USER_factors
            self.ITEM_factors = ITEM_factors
        
    def _update_best_model(self):
        self.USER_factors_best = self.USER_factors.copy()
        self.ITEM_factors_best = self.ITEM_factors.copy()
        
    def _prepare_model_for_validation(self):
        pass

In [15]:
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])

early_stopping_params = {
    "validation_every_n": 1,
    "stop_on_validation": True,
    "evaluator_object": evaluator,
    "lower_validations_allowed": 3,
    "validation_metric": "MAP",
    "epochs": 10
} 

recommender = LightGCNRecommender(URM_train)
recommender.fit(init_std=0.1, **early_stopping_params)

EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
LightGCNRecommender: URM Detected 627 ( 4.8%) users with no interactions.
LightGCNRecommender: URM Detected 247 ( 1.1%) items with no interactions.
LightGCNRecommender: Validation begins...
EvaluatorHoldout: Processed 10456 (100.0%) in 7.46 sec. Users per second: 1402
LightGCNRecommender: CUTOFF: 10 - PRECISION: 0.0001243, PRECISION_RECALL_MIN_DEN: 0.0002219, RECALL: 0.0001669, MAP: 0.0000451, MAP_MIN_DEN: 0.0000613, MRR: 0.0004507, NDCG: 0.0001702, F1: 0.0001425, HIT_RATE: 0.0012433, ARHR_ALL_HITS: 0.0004507, NOVELTY: 0.0076310, AVERAGE_POPULARITY: 0.0023424, DIVERSITY_MEAN_INTER_LIST: 0.8648407, DIVERSITY_HERFINDAHL: 0.9864758, COVERAGE_ITEM: 0.0155278, COVERAGE_ITEM_HIT: 0.0004475, ITEMS_IN_GT: 0.8004206, COVERAGE_USER: 0.8028256, COVERAGE_USER_HIT: 0.0009982, USERS_IN_GT: 0.8028256, DIVERSITY_GINI: 0.0037267, SHANNON_ENTROPY: 6.7841391, RATIO_DIVERSITY_HERFINDAHL: 0.9868326, RATIO_DIVERSITY_GINI: 

In [16]:
recommender = LightGCNRecommender(URM_train)
recommender.fit(init_std=0.1, l2_user=10, l2_item=10, **early_stopping_params)

LightGCNRecommender: URM Detected 627 ( 4.8%) users with no interactions.
LightGCNRecommender: URM Detected 247 ( 1.1%) items with no interactions.
LightGCNRecommender: Validation begins...
EvaluatorHoldout: Processed 10456 (100.0%) in 7.25 sec. Users per second: 1442
LightGCNRecommender: CUTOFF: 10 - PRECISION: 0.0000765, PRECISION_RECALL_MIN_DEN: 0.0002019, RECALL: 0.0001816, MAP: 0.0000375, MAP_MIN_DEN: 0.0001302, MRR: 0.0003746, NDCG: 0.0001907, F1: 0.0001077, HIT_RATE: 0.0007651, ARHR_ALL_HITS: 0.0003746, NOVELTY: 0.0076576, AVERAGE_POPULARITY: 0.0022129, DIVERSITY_MEAN_INTER_LIST: 0.8724980, DIVERSITY_HERFINDAHL: 0.9872415, COVERAGE_ITEM: 0.0157516, COVERAGE_ITEM_HIT: 0.0003580, ITEMS_IN_GT: 0.8004206, COVERAGE_USER: 0.8028256, COVERAGE_USER_HIT: 0.0006143, USERS_IN_GT: 0.8028256, DIVERSITY_GINI: 0.0039045, SHANNON_ENTROPY: 6.8545242, RATIO_DIVERSITY_HERFINDAHL: 0.9875985, RATIO_DIVERSITY_GINI: 0.0112661, RATIO_SHANNON_ENTROPY: 0.5280459, RATIO_AVERAGE_POPULARITY: 0.0184527, RATI

In [17]:
early_stopping_params = {
    "validation_every_n": 1,
    "stop_on_validation": True,
    "evaluator_object": evaluator,
    "lower_validations_allowed": 10,
    "validation_metric": "MAP",
    "epochs": 500
}


def objective_function_lightgcn(trial):
    recommender_instance = LightGCNRecommender(URM_train)
    recommender_instance.fit(
        num_layers = trial.suggest_int('num_layers', 1, 10),
        embedding_size = trial.suggest_int('embedding_size', 5, 200, log=True),
        learning_rate = trial.suggest_float('learning_rate', 1e-6, 1e-2, log=True),
        init_std = trial.suggest_float('init_std', 1e-3, 1, log=True),
        optimizer = trial.suggest_categorical('optimizer', ["adam", "sgd", "rmsprop", "adagrad"]),
        loss = trial.suggest_categorical('loss', ["mse", "xentropy"]),
        l2_user = trial.suggest_float('l2_user', 1e-5, 100, log=True),
        l2_item = trial.suggest_float('l2_item', 1e-5, 100, log=True),
        **early_stopping_params
    )
    
    epochs = recommender_instance.get_early_stopping_final_epochs_dict()["epochs"]
    trial.set_user_attr("epochs", epochs) 
        
    result_df, _ = evaluator.evaluateRecommender(recommender_instance)
    
    return result_df.loc[10, "MAP"]

In [18]:
if config['tune_parameters']:
    study = optuna.create_study(direction='maximize', study_name='tuning_lightgcn_v1',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)
    study.optimize(objective_function_lightgcn, n_trials=30)

[I 2023-12-17 22:02:56,943] A new study created in RDB with name: tuning_lightgcn_v1


LightGCNRecommender: URM Detected 627 ( 4.8%) users with no interactions.
LightGCNRecommender: URM Detected 247 ( 1.1%) items with no interactions.
LightGCNRecommender: Validation begins...
EvaluatorHoldout: Processed 10456 (100.0%) in 7.49 sec. Users per second: 1396
LightGCNRecommender: CUTOFF: 10 - PRECISION: 0.0001243, PRECISION_RECALL_MIN_DEN: 0.0002163, RECALL: 0.0001797, MAP: 0.0000306, MAP_MIN_DEN: 0.0000457, MRR: 0.0003059, NDCG: 0.0001464, F1: 0.0001470, HIT_RATE: 0.0012433, ARHR_ALL_HITS: 0.0003059, NOVELTY: 0.0076203, AVERAGE_POPULARITY: 0.0037733, DIVERSITY_MEAN_INTER_LIST: 0.4096748, DIVERSITY_HERFINDAHL: 0.9409636, COVERAGE_ITEM: 0.0059963, COVERAGE_ITEM_HIT: 0.0005370, ITEMS_IN_GT: 0.8004206, COVERAGE_USER: 0.8028256, COVERAGE_USER_HIT: 0.0009982, USERS_IN_GT: 0.8028256, DIVERSITY_GINI: 0.0008969, SHANNON_ENTROPY: 4.6296402, RATIO_DIVERSITY_HERFINDAHL: 0.9413039, RATIO_DIVERSITY_GINI: 0.0025879, RATIO_SHANNON_ENTROPY: 0.3566495, RATIO_AVERAGE_POPULARITY: 0.0314643, RATI

[I 2023-12-17 22:10:56,729] Trial 0 finished with value: 0.011604842970087756 and parameters: {'num_layers': 5, 'embedding_size': 22, 'learning_rate': 0.0027596058180351392, 'init_std': 0.005687960958969734, 'optimizer': 'sgd', 'loss': 'xentropy', 'l2_user': 5.931421176914334, 'l2_item': 0.0029409714684325993}. Best is trial 0 with value: 0.011604842970087756.


LightGCNRecommender: URM Detected 627 ( 4.8%) users with no interactions.
LightGCNRecommender: URM Detected 247 ( 1.1%) items with no interactions.
LightGCNRecommender: Validation begins...
EvaluatorHoldout: Processed 10456 (100.0%) in 7.28 sec. Users per second: 1437
LightGCNRecommender: CUTOFF: 10 - PRECISION: 0.0001148, PRECISION_RECALL_MIN_DEN: 0.0002997, RECALL: 0.0002682, MAP: 0.0000344, MAP_MIN_DEN: 0.0001566, MRR: 0.0003439, NDCG: 0.0002431, F1: 0.0001608, HIT_RATE: 0.0011477, ARHR_ALL_HITS: 0.0003439, NOVELTY: 0.0076944, AVERAGE_POPULARITY: 0.0021700, DIVERSITY_MEAN_INTER_LIST: 0.8015610, DIVERSITY_HERFINDAHL: 0.9801484, COVERAGE_ITEM: 0.0156173, COVERAGE_ITEM_HIT: 0.0004922, ITEMS_IN_GT: 0.8004206, COVERAGE_USER: 0.8028256, COVERAGE_USER_HIT: 0.0009214, USERS_IN_GT: 0.8028256, DIVERSITY_GINI: 0.0028583, SHANNON_ENTROPY: 6.3659368, RATIO_DIVERSITY_HERFINDAHL: 0.9805029, RATIO_DIVERSITY_GINI: 0.0082474, RATIO_SHANNON_ENTROPY: 0.4904070, RATIO_AVERAGE_POPULARITY: 0.0180949, RATI

[I 2023-12-17 22:15:36,040] Trial 1 finished with value: 3.438824522412893e-05 and parameters: {'num_layers': 3, 'embedding_size': 94, 'learning_rate': 1.76272817817296e-06, 'init_std': 0.4912389510018123, 'optimizer': 'rmsprop', 'loss': 'xentropy', 'l2_user': 0.2315497063399144, 'l2_item': 0.006035647115420991}. Best is trial 0 with value: 0.011604842970087756.


...

[I 2023-12-18 06:44:38,959] Trial 27 finished with value: 0.011598808612963111 and parameters: {'num_layers': 8, 'embedding_size': 7, 'learning_rate': 0.0047331926678406055, 'init_std': 0.022603090046401375, 'optimizer': 'adagrad', 'loss': 'xentropy', 'l2_user': 0.10438175540595224, 'l2_item': 3.254794436915987}. Best is trial 20 with value: 0.011619788440752425.


LightGCNRecommender: URM Detected 627 ( 4.8%) users with no interactions.
LightGCNRecommender: URM Detected 247 ( 1.1%) items with no interactions.
LightGCNRecommender: Validation begins...
EvaluatorHoldout: Processed 10456 (100.0%) in 6.60 sec. Users per second: 1585
LightGCNRecommender: CUTOFF: 10 - PRECISION: 0.0288829, PRECISION_RECALL_MIN_DEN: 0.0528925, RECALL: 0.0452377, MAP: 0.0115840, MAP_MIN_DEN: 0.0207209, MRR: 0.0905154, NDCG: 0.0443539, F1: 0.0352560, HIT_RATE: 0.2298202, ARHR_ALL_HITS: 0.1020450, NOVELTY: 0.0039196, AVERAGE_POPULARITY: 0.7189359, DIVERSITY_MEAN_INTER_LIST: 0.1965568, DIVERSITY_HERFINDAHL: 0.9196538, COVERAGE_ITEM: 0.0015215, COVERAGE_ITEM_HIT: 0.0008950, ITEMS_IN_GT: 0.8004206, COVERAGE_USER: 0.8028256, COVERAGE_USER_HIT: 0.1845055, USERS_IN_GT: 0.8028256, DIVERSITY_GINI: 0.0005793, SHANNON_ENTROPY: 3.8177262, RATIO_DIVERSITY_HERFINDAHL: 0.9199864, RATIO_DIVERSITY_GINI: 0.0016714, RATIO_SHANNON_ENTROPY: 0.2941028, RATIO_AVERAGE_POPULARITY: 5.9949573, RATI

[I 2023-12-18 06:49:11,054] Trial 28 finished with value: 0.011584041516376969 and parameters: {'num_layers': 9, 'embedding_size': 15, 'learning_rate': 0.0018117154072852543, 'init_std': 0.008814206236927464, 'optimizer': 'adam', 'loss': 'xentropy', 'l2_user': 0.017763960271676236, 'l2_item': 1.0773965416636553}. Best is trial 20 with value: 0.011619788440752425.


LightGCNRecommender: URM Detected 627 ( 4.8%) users with no interactions.
LightGCNRecommender: URM Detected 247 ( 1.1%) items with no interactions.
LightGCNRecommender: Validation begins...
EvaluatorHoldout: Processed 10456 (100.0%) in 6.26 sec. Users per second: 1670
LightGCNRecommender: CUTOFF: 10 - PRECISION: 0.0260042, PRECISION_RECALL_MIN_DEN: 0.0445355, RECALL: 0.0372925, MAP: 0.0098168, MAP_MIN_DEN: 0.0164676, MRR: 0.0771241, NDCG: 0.0372829, F1: 0.0306418, HIT_RATE: 0.2090666, ARHR_ALL_HITS: 0.0867602, NOVELTY: 0.0047539, AVERAGE_POPULARITY: 0.5768483, DIVERSITY_MEAN_INTER_LIST: 0.4780492, DIVERSITY_HERFINDAHL: 0.9478003, COVERAGE_ITEM: 0.0100237, COVERAGE_ITEM_HIT: 0.0009397, ITEMS_IN_GT: 0.8004206, COVERAGE_USER: 0.8028256, COVERAGE_USER_HIT: 0.1678440, USERS_IN_GT: 0.8028256, DIVERSITY_GINI: 0.0011380, SHANNON_ENTROPY: 4.9118296, RATIO_DIVERSITY_HERFINDAHL: 0.9481431, RATIO_DIVERSITY_GINI: 0.0032837, RATIO_SHANNON_ENTROPY: 0.3783882, RATIO_AVERAGE_POPULARITY: 4.8101379, RATI

[I 2023-12-18 06:55:40,805] Trial 29 finished with value: 0.01150074461689798 and parameters: {'num_layers': 3, 'embedding_size': 20, 'learning_rate': 3.4555950405524256e-05, 'init_std': 0.003515160596783922, 'optimizer': 'rmsprop', 'loss': 'xentropy', 'l2_user': 0.4967604981350461, 'l2_item': 0.3128265762880623}. Best is trial 20 with value: 0.011619788440752425.


In [19]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

In [20]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_param_importances(study)
    fig.show()

In [21]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_slice(study)
    fig.show()

In [22]:
if config['tune_parameters']:
    with open('/kaggle/working/best_params_lightgcn.json', 'w') as params_file:
        params = study.best_params
        params['epochs'] = study.best_trial.user_attrs['epochs']
        json.dump(params, params_file)
        
    if config['save_github']: 
        upload_file(
            '/kaggle/working/best_params_lightgcn.json', 
            'tuning_results/best_params_lightgcn.json', 
            'LightGCN tuning results (from kaggle notebook)'
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/best_params_lightgcn.json',
        '/kaggle/working/best_params_lightgcn.json'
    )

In [23]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        'tuning_results/tuning_lightgcn.db',
        'LightGCN tuning db updated results (from kaggle notebook)'
    )